In [1]:
import os

import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold

from tqdm import tqdm
from random import choices
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

2023-06-04 12:47:52.510291: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-04 12:47:53.532927: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-04 12:47:53.533015: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-06-04 12:47:53.533026: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [2]:
# tf setup
print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

MIXED_PRECISION = False
XLA_ACCELERATE = True

if MIXED_PRECISION:
    from tensorflow.keras.mixed_precision import experimental as mixed_precision
    if tpu: policy = tf.keras.mixed_precision.experimental.Policy('mixed_bfloat16')
    else: policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
    mixed_precision.set_policy(policy)
    print('Mixed precision enabled')

if XLA_ACCELERATE:
    tf.config.optimizer.set_jit(True)
    print('Accelerated Linear Algebra enabled')

Tensorflow version 2.11.0
Accelerated Linear Algebra enabled


In [ ]:
CV_STRATEGY = 'StratifiedGroupKFold' # GroupKFold, PurgedGroupTimeSeriesSplit
SEED = 2021
# START_DATE = 86
FOLDS = 5

DEBUG = False

In [3]:
from sklearn.model_selection import KFold
import os
import sys
import numpy as np
import pandas as pd
import random
from collections import Counter, defaultdict
from sklearn import model_selection

In [22]:
# from scipy import stats
# from scipy.stats import norm, skew
# all_data = pd.read_csv('../data/forauto.csv')
# useless_cols = ['Category.1', 'Concept.1', 'Subcategory.1', 'Alltags','Title','img_file']
# cate_cols = ['Uid', 'Category', 'Subcategory', 'Concept', 'Mediatype', 'hour', 'day', 'weekday', 'week_hour',
#              'Geoaccuracy', 'ispro' , 'Ispublic', 'img_model']
# useless_cols += cate_cols
# all_data = all_data.drop(useless_cols,axis=1)
# # alltags = all_data[['Alltags','label']]
# # alltags.columns = ['excerpt', 'target']
# rob = pd.read_csv('./preRoberta_fea.csv')
# all_data = pd.concat([all_data,rob],axis=1)
# columns = ['Title_len', 'Title_number', 'Alltags_len', 'Alltags_number', 'photo_count', 'totalTags', 'totalGeotagged', 'totalFaves',
#           'totalInGroup','photoCount','meanView', 'meanTags', 'meanFaves', 'followerCount','followingCount']

# skew_features = all_data[columns].apply(lambda x: skew(x)).sort_values(ascending=False)
# high_skew = skew_features[abs(skew_features) > 0.75]
# skew_index = high_skew.index
# for i in skew_index:
#     all_data[i] = np.log1p(all_data[i])
    
# train = all_data[:-180581]
# test = all_data[-180581:]

# # test = test.reset_index(drop=True)
# y = train['label'].values
# X = train.drop(['label'],axis=1).values

# # alltags

In [4]:
from scipy import stats
from scipy.stats import norm, skew
from sklearn.preprocessing import StandardScaler
all_data = pd.read_csv('../data/forauto.csv')
useless_cols = ['year_weekday', 'Longitude','Uid_count','img_length','img_width', 'pixel','Category.1', 
                'Concept.1', 'Subcategory.1', 'Alltags','Title','img_file','firstdate','firstweek', 'firstmonth', 
                'firstdatetaken', 'firstdatetakenweek','firstdatetakenmonth','totalViews']
cate_cols = ['Uid', 'Category', 'Latitude', 'Geoaccuracy', 'Subcategory', 'Concept', 'Mediatype', 
             'hour', 'day', 'weekday', 'week_hour','Geoaccuracy', 'ispro' , 'Ispublic', 'img_model']
useless_cols += cate_cols
all_data = all_data.drop(useless_cols,axis=1)
# alltags = all_data[['Alltags','label']]
# alltags.columns = ['excerpt', 'target']
rob = pd.read_csv('./preRoberta_fea.csv')
all_data = pd.concat([all_data,rob],axis=1)
columns = ['Title_len', 'Title_number', 'Alltags_len', 'Alltags_number', 
           'photo_count', 'totalTags', 'totalGeotagged', 'totalFaves',
          'totalInGroup','photoCount','meanView', 'meanTags', 'meanFaves', 'followerCount','followingCount']

# skew_features = all_data[columns].apply(lambda x: skew(x)).sort_values(ascending=False)
# high_skew = skew_features[abs(skew_features) > 0.75]
# skew_index = high_skew.index
for i in columns:
    all_data[i] = np.log1p(all_data[i])
# numerical_cols = []
for c in columns:
    prep = StandardScaler()
    all_data[c] = prep.fit_transform(all_data[[c]])
    # train[c] = prep.fit_transform(train[[c]])
    # test[c] = prep.transform(test[[c]])
    
train = all_data[:-180581]
test = all_data[-180581:]

test = test.reset_index(drop=True)

y = train[['label']].values
X = train.drop(['label'],axis=1).values
df_test = test.drop(['label'],axis=1).values


In [5]:
def create_autoencoder(input_dim,output_dim,noise=0.01):
    i = tf.keras.layers.Input(input_dim)
    encoded = tf.keras.layers.BatchNormalization()(i)
    encoded = tf.keras.layers.GaussianNoise(noise)(encoded)
    encoded = tf.keras.layers.Dense(64,activation='relu')(encoded)
    decoded = tf.keras.layers.Dropout(0.1)(encoded)
    decoded = tf.keras.layers.Dense(input_dim,name='decoded')(decoded)
    x = tf.keras.layers.Dense(32,activation='relu')(decoded)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(output_dim,activation='linear',name='label_output')(x)
    
    encoder = tf.keras.models.Model(inputs=i,outputs=encoded)
    autoencoder = tf.keras.models.Model(inputs=i,outputs=[decoded,x])
    
    autoencoder.compile(optimizer=tf.keras.optimizers.Adam(0.001), 
                        loss={'decoded':'mae','label_output':'mae'})
    return autoencoder, encoder

In [11]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 
def create_model(input_dim, output_dim, encoder):
    # input
    inputs = tf.keras.layers.Input(input_dim)
    
    x = encoder(inputs)
    x = tf.keras.layers.Concatenate()([x,inputs]) #use both raw and encoded features
    
    # normalize
    x = tf.keras.layers.BatchNormalization()(x)
    
    # 1dcnn
    x = tf.keras.layers.Dense(4096, activation='relu')(x)
    x = tf.keras.layers.Reshape((256, 16))(x)
    x = tf.keras.layers.Conv1D(filters=16,
                      kernel_size=7,
                      strides=1,
                      activation='relu')(x)
    x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
    x = tf.keras.layers.Flatten()(x)
    
    # ffn
    for i in range(2):
        x = tf.keras.layers.Dense(256 // (2 ** i), activation='relu')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.GaussianNoise(0.01)(x)
        x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(output_dim, activation='relu')(x)
    
    model = tf.keras.models.Model(inputs=inputs,outputs=x)
    
    # compile
    opt = tfa.optimizers.RectifiedAdam(learning_rate=1e-03)
    opt = tfa.optimizers.SWA(opt)
    loss = tf.keras.losses.MeanAbsoluteError()
    # loss = root_mean_squared_error
    model.compile(optimizer=opt, 
                  loss=loss, 
                  metrics=[tf.keras.metrics.MeanAbsoluteError(name = 'root_mean_squared_error')])
    return model

In [7]:
autoencoder, encoder = create_autoencoder(X.shape[-1],y.shape[-1],noise=0.1)
autoencoder.fit(X,(X,y),
                epochs=1000,
                batch_size=256, 
                validation_split=0.1,
                callbacks=[tf.keras.callbacks.EarlyStopping('val_loss',patience=10,restore_best_weights=True)])
# autoencoder.predict
encoder.save_weights('./encoder.hdf5')
encoder.trainable = False

2023-06-04 12:57:38.044890: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-04 12:57:38.665922: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22279 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:1a:00.0, compute capability: 8.6


Epoch 1/1000


2023-06-04 12:57:45.471454: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1780] (One-time warning): Not using XLA:CPU for cluster.

If you want XLA:CPU, do one of the following:

 - set the TF_XLA_FLAGS to include "--tf_xla_cpu_global_jit", or
 - set cpu_global_jit to true on this session's OptimizerOptions, or
 - use experimental_jit_scope, or
 - use tf.function(jit_compile=True).

To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a
proper command-line flag, not via TF_XLA_FLAGS).
2023-06-04 12:57:46.643460: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0xbb32b7b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-06-04 12:57:46.643538: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-06-04 12:57:46.662848: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env 

1075/1075 [==============================] - 20s 15ms/step - loss: 2.2772 - decoded_loss: 0.1493 - label_output_loss: 2.1279 - val_loss: 1.8811 - val_decoded_loss: 0.0949 - val_label_output_loss: 1.7862
Epoch 2/1000
1075/1075 [==============================] - 8s 7ms/step - loss: 1.2006 - decoded_loss: 0.0867 - label_output_loss: 1.1139 - val_loss: 1.7024 - val_decoded_loss: 0.0723 - val_label_output_loss: 1.6301
Epoch 3/1000
1075/1075 [==============================] - 7s 6ms/step - loss: 1.1099 - decoded_loss: 0.0734 - label_output_loss: 1.0365 - val_loss: 1.8924 - val_decoded_loss: 0.0675 - val_label_output_loss: 1.8249
Epoch 4/1000
1075/1075 [==============================] - 7s 6ms/step - loss: 1.0618 - decoded_loss: 0.0697 - label_output_loss: 0.9921 - val_loss: 1.8861 - val_decoded_loss: 0.0660 - val_label_output_loss: 1.8200
Epoch 5/1000
1075/1075 [==============================] - 7s 6ms/step - loss: 1.0282 - decoded_loss: 0.0678 - label_output_loss: 0.9604 - val_loss: 1.9392 

In [12]:
from sklearn.model_selection import StratifiedKFold, KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=2020)
fold = 0
y_preds = []
models = []
for train_indices, test_indices in kfold.split(X, y):
# for fold, (train_indices, test_indices) in tqdm(enumerate(splits)):
    X_train, X_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]
    
    # model
    tf.keras.backend.clear_session()
    model = create_model(X.shape[-1], y.shape[-1], encoder)
    
    # callbacks
    er = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True, monitor='val_loss')
    ReduceLR = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=8, verbose=1)
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=f'./model_{2020}_{fold}.hdf5', save_weights_only=True, verbose=0, monitor='root_mean_squared_error', save_best_only=True)
    nn_callbacks = [er, ReduceLR, model_checkpoint_callback]
    
#     valid_pred = model.predict(X[valid_idx, :]).reshape(1, -1)[0]
#     valid_mse = mean_squared_error(y[valid_idx], valid_pred)
#     valid_mae = mean_absolute_error(y[valid_idx], valid_pred)
#     valid_src = stats.spearmanr(y[valid_idx], valid_pred)[0]

#     print(f'Fold-{fold}: RMSE={valid_mse}, MAE={valid_mae},SRC={valid_src} ')
    
#     y_pred = model.predict(test_df).reshape(1, -1)[0]
#     y_preds.append(y_pred)
    # models.append(model)
    model.save(f"./dae_1dcnn_f{fold}.h5")
    # rmse_lst.append(valid_mse)
    
    # fit
    model.fit(X_train, y_train, validation_data=(X_test,y_test), 
              epochs=192, batch_size=2048, callbacks=nn_callbacks)
    y_pred = model.predict(df_test).reshape(1, -1)[0]
    y_preds.append(y_pred)
    # model.predict(df_test)
    # models.append(model)
    fold += 1

Epoch 1/192
120/120 [==============================] - 8s 29ms/step - loss: 3.4140 - root_mean_squared_error: 3.4140 - val_loss: 2.2752 - val_root_mean_squared_error: 2.2752 - lr: 0.0010
Epoch 2/192
120/120 [==============================] - 4s 34ms/step - loss: 2.1518 - root_mean_squared_error: 2.1518 - val_loss: 1.9424 - val_root_mean_squared_error: 1.9424 - lr: 0.0010
Epoch 3/192
120/120 [==============================] - 2s 18ms/step - loss: 1.8133 - root_mean_squared_error: 1.8133 - val_loss: 2.1170 - val_root_mean_squared_error: 2.1170 - lr: 0.0010
Epoch 4/192
120/120 [==============================] - 2s 18ms/step - loss: 1.5780 - root_mean_squared_error: 1.5780 - val_loss: 1.8669 - val_root_mean_squared_error: 1.8669 - lr: 0.0010
Epoch 5/192
120/120 [==============================] - 2s 20ms/step - loss: 1.3985 - root_mean_squared_error: 1.3985 - val_loss: 1.4180 - val_root_mean_squared_error: 1.4180 - lr: 0.0010
Epoch 6/192
120/120 [==============================] - 2s 19ms/st

In [13]:
y_preds

[array([ 8.81501  , 10.458895 ,  9.324839 , ...,  7.6694593,  9.755777 ,
         5.697639 ], dtype=float32),
 array([ 9.409686 , 10.5654955, 10.604354 , ...,  8.048788 ,  9.741979 ,
         6.3828497], dtype=float32),
 array([ 8.611053 , 11.598322 ,  9.336557 , ...,  7.656889 ,  9.716085 ,
         7.8818555], dtype=float32),
 array([ 8.94466 , 10.535761,  9.877227, ...,  7.516124,  9.081009,
         7.525093], dtype=float32),
 array([ 7.6713247, 11.206317 ,  9.815075 , ...,  8.012328 , 10.056163 ,
         6.2926197], dtype=float32)]

In [21]:
pd.DataFrame(valid_ans,columns=['dae_nn_mean']).to_csv('./dae_nn_mean.csv',header=True,index=None)

In [20]:
valid_ans = np.mean(y_preds, axis=0)
valid_ans

array([ 8.690348 , 10.872957 ,  9.79161  , ...,  7.7807183,  9.670203 ,
        6.756011 ], dtype=float32)

In [ ]:
# ['Uid', 'Uid_count', 'Category', 'Subcategory', 'Concept', 'Title_len',
#        'Title_number', 'Alltags_len', 'Alltags_number', 'img_length',
#        'img_width', 'pixel', 'img_model', 'svd_mode_t_0', 'svd_mode_t_1',
#        'svd_mode_t_2', 'svd_mode_t_3', 'svd_mode_t_4', 'svd_mode_t_5',
#        'svd_mode_t_6', 'svd_mode_t_7', 'svd_mode_t_8', 'svd_mode_t_9',
#        'svd_mode_t_10', 'svd_mode_t_11', 'svd_mode_t_12', 'svd_mode_t_13',
#        'svd_mode_t_14', 'svd_mode_t_15', 'svd_mode_t_16', 'svd_mode_t_17',
#        'svd_mode_t_18', 'svd_mode_t_19', 'svd_mode_0', 'svd_mode_1',
#        'svd_mode_2', 'svd_mode_3', 'svd_mode_4', 'svd_mode_5', 'svd_mode_6',
#        'svd_mode_7', 'svd_mode_8', 'svd_mode_9', 'svd_mode_10', 'svd_mode_11',
#        'svd_mode_12', 'svd_mode_13', 'svd_mode_14', 'svd_mode_15',
#        'svd_mode_16', 'svd_mode_17', 'svd_mode_18', 'svd_mode_19', 'Mediatype',
#        'hour', 'day', 'weekday', 'week_hour', 'year_weekday', 'Longitude',
#        'Latitude', 'Geoaccuracy', 'photo_count', 'ispro', 'firstdate',
#        'firstweek', 'firstmonth', 'firstdatetaken', 'firstdatetakenweek',
#        'firstdatetakenmonth', 'totalViews', 'totalTags', 'totalGeotagged',
#        'totalFaves', 'totalInGroup', 'photoCount', 'meanView', 'meanTags',
#        'meanFaves', 'followerCount', 'followingCount', 'Ispublic', 'label',
#        'img_file', 'Category.1', 'Concept.1', 'Subcategory.1', 'Alltags',
#        'Title']

In [8]:
# # seed_everything()

# #kf = StratifiedKFold(5, shuffle=True, random_state=GCF.SEED)
# # kf = TimeSeriesSplit(n_splits=GCF.N_FOLDS, max_train_size=GCF.N_TRAIN)
# from scipy import stats
# from sklearn.metrics import f1_score, mean_absolute_error, mean_squared_error
# from scipy import stats
# rmse_lst, score_lst = [], []
# #oof = np.zeros((len(y),))
# #for fold, (train_idx, valid_idx) in enumerate(kf.split(X, investment_id)):

# kfold = KFold(n_splits=5, shuffle=True, random_state=2020)
# fold = 0

# for train_idx, valid_idx in kfold.split(X, y):
#     model = create_model()

#     early_stopping = keras.callbacks.EarlyStopping(
#         monitor='root_mean_squared_error',
#         patience=GCF.EARLY_STOPPING_PATIENCE,
#         min_delta=GCF.EARLY_STOPPING_MIN_DELTA,
#         restore_best_weights=True,
#     )
#     reduce_lr = ReduceLROnPlateau(
#                         #monitor='val_loss',
#                         monitor='root_mean_squared_error',
#                         factor=0.5,
#                         patience=3,
#                         min_lr=1e-5,
#                         verbose=1
#     )

#     model.fit(
#         X[train_idx, :], y[train_idx],
#         validation_data=(X[valid_idx, :], y[valid_idx]),
#         batch_size=GCF.BATCH_SIZE,
#         epochs=GCF.N_EPOCHS,
#         callbacks=[early_stopping, reduce_lr],
#     )
    
#     valid_pred = model.predict(X[valid_idx, :]).reshape(1, -1)[0]
#     valid_mse = mean_squared_error(y[valid_idx], valid_pred)
#     valid_mae = mean_absolute_error(y[valid_idx], valid_pred)
#     valid_src = stats.spearmanr(y[valid_idx], valid_pred)[0]
#     print(f'Fold-{fold}: RMSR={valid_mse}, MAE={valid_mae},SRC={valid_src} ')
# #     oof_train[valid_index] = model.predict(X_val).reshape(1, -1)[0]
# #     y_pred = model.predict(X_test).reshape(1, -1)[0]

# #     y_preds.append(y_pred)
#     models.append(model)
#     model.save(f"./ump_1dcnn_f{fold}.h5")
#     rmse_lst.append(valid_mse)
#     fold += 1